This notebook focuses on calculating the missed points for each team by week

In [1]:
from dotenv import load_dotenv
import os
from espn_api.football import League
from util.player_tiers_helper import get_player_id_to_player_dict
from util.player_helper import get_team_actual_lineup_given_week, get_team_actual_lineup_given_week_total_points, get_player_points_or_projected_points, get_team_best_lineup_given_week, get_team_best_lineup_given_week_total_points, get_team_actual_lineup_player_ids_given_week, get_team_best_lineup_player_ids_given_week

/Users/sahilkancherla/Documents/ESPNFFAdvancedStats/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'util'

In [ ]:
load_dotenv()

league_id = os.getenv('LEAGUE_ID')
swid = os.getenv('SWID')
espn_s2 = os.getenv('ESPN_S2')

In [ ]:
year = 2024

In [ ]:
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)

In [ ]:
week = 1

In [ ]:
def get_ids_of_starters(league, team_name, week, ignore_slots = ["BE", "IR"]):
    for box_score in league.box_scores(week = week):
        if box_score.home_team.team_name == team_name:
            lineup = box_score.home_lineup
        elif box_score.away_team.team_name == team_name:
            lineup = box_score.away_lineup
        else:
            continue

        player_ids = []

        for player in lineup:
            if not ignore_slots.__contains__(player.lineupSlot):
                player_ids.append(player.playerId)
        return player_ids

    return None

In [ ]:
# def get_substitutions_for_best_lineup(league, team_name, week):
    
#     changes = []

#     actual_lineup = get_team_actual_lineup_given_week(league, team_name, week)
#     best_lineup = get_team_best_lineup_given_week(league, team_name, week)

#     for position in best_lineup.keys():
#         best_players_at_pos = []
#         actual_players_at_pos = []

#         for boxPlayer in best_lineup[position]:
#             best_players_at_pos.append((boxPlayer.playerId, boxPlayer))

#         for boxPlayer in actual_lineup[position]:
#             actual_players_at_pos.append((boxPlayer.playerId, boxPlayer))

#         diff_best_players_at_pos = [player for player in best_players_at_pos if player[0] not in [p[0] for p in actual_players_at_pos]]
#         diff_actual_players_at_pos = [player for player in actual_players_at_pos if player[0] not in [p[0] for p in best_players_at_pos]]
        
#         sorted_diff_best_players_at_pos = sorted(diff_best_players_at_pos, key=lambda player: get_player_points_or_projected_points(player[1]), reverse=True)
#         sorted_diff_actual_players_at_pos = sorted(diff_actual_players_at_pos, key=lambda player: get_player_points_or_projected_points(player[1]), reverse=True)

#         actual_ids = get_ids_of_starters(league, team_name, week)

#         for i, better_player in enumerate(sorted_diff_best_players_at_pos):
#             actual_player = sorted_diff_actual_players_at_pos[i][1]

#             if better_player[0] not in actual_ids:
#                 changes.append((better_player[1], actual_player, position))

#     return changes    
        



In [ ]:
# def assign_players_to_slots(empty_slots, players):
#     assigned_slots = {}

#     for slot in empty_slots:
#         for player in players:
#             if slot in player.eligibleSlots:
#                 assigned_slots[slot] = player
                
#                 players.remove(player)
#                 break
#         else:
#             assigned_slots[slot] = None
    
#     return assigned_slots

# def get_substitutions_for_best_lineup(league, team_name, week):
    
#     changes = []

#     actual_lineup = get_team_actual_lineup_given_week(league, team_name, week)
#     best_lineup = get_team_best_lineup_given_week(league, team_name, week)

#     best_players = []
#     actual_players = []

#     for position in best_lineup.keys():

#         for boxPlayer in best_lineup[position]:
#             best_players.append((boxPlayer.playerId, boxPlayer))

#         for boxPlayer in actual_lineup[position]:
#             actual_players.append((boxPlayer.playerId, boxPlayer))

#     unique_best_players = [player for player in best_players if player[0] not in [p[0] for p in actual_players]]
#     unique_actual_players = [player for player in actual_players if player[0] not in [p[0] for p in best_players]]

#     # find the positions that need to be filled by new players

#     best_players = []

#     for player in unique_best_players:
#         best_players.append(player[1])

#     empty_slots = []
#     for player in unique_actual_players:
#         empty_slots.append(player[1].slot_position)

#     print('unique_best_players', unique_best_players)
#     print('unique_actual_players', unique_actual_players)
#     print('empty_slots', empty_slots)
#     assigned_slots = assign_players_to_slots(empty_slots, best_players)

#     print("Assigned_slots", assigned_slots)
    
#     changes = []
#     for i, slot in enumerate(empty_slots):
#         changes.append((slot, assigned_slots[slot], unique_actual_players[i][1]))

#     return changes

    

In [ ]:
def assign_players_to_slots(empty_slots, players):
    assigned_slots = {}

    for slot in empty_slots:
        for player in players:
            if slot in player.eligibleSlots:
                assigned_slots[slot] = player
                
                players.remove(player)
                break
        else:
            assigned_slots[slot] = None
    
    return assigned_slots

def remove_common_elements_from_dicts(dict1, dict2):
    for key in dict1.keys():
        if key in dict2:
            list1 = dict1[key]
            list2 = dict2[key]
            
            common_elements = set(list1) & set(list2)
            
            dict1[key] = [item for item in list1 if item not in common_elements]
            dict2[key] = [item for item in list2 if item not in common_elements]
    
    dict1 = {key: value for key, value in dict1.items() if value}
    dict2 = {key: value for key, value in dict2.items() if value}
    
    return dict1, dict2

def get_substitutions_for_best_lineup(league, team_name, week):
    
    actual_lineup = get_team_actual_lineup_player_ids_given_week(league, team_name, week)
    best_lineup = get_team_best_lineup_player_ids_given_week(league, team_name, week)

    new_actual_lineup, new_best_lineup = remove_common_elements_from_dicts(actual_lineup, best_lineup)

    print(new_actual_lineup)
    print(new_best_lineup)
    

In [ ]:
get_substitutions_for_best_lineup(league, "How I Metcalf Your Mother", 1)

{'WR': [4047650], 'RB': [4047365, 4430737], 'RB/WR/TE': [3045147], 'QB': [3915511]}
{'QB': [2577417], 'RB': [3045147, 4241474], 'WR': [4362249], 'RB/WR/TE': [4430737]}


In [ ]:
for team in league.teams:
    team_name = team.team_name
    if team_name == 'Unknown' or team_name == "League Median":
        continue

    actual_pts = get_team_actual_lineup_given_week_total_points(league, team_name, week, ["BE", "IR"])
    actual_lineup = get_team_actual_lineup_given_week(league, team_name, week, ["BE", "IR"])
    best_pts = get_team_best_lineup_given_week_total_points(league, team_name, week, ["BE", "IR"])
    best_lineup = get_team_best_lineup_given_week(league, team_name, week, ["BE", "IR"])
        
    print("Name: ", team_name)
    print("Missed points: ", abs(max(round(best_pts - actual_pts, 2), 0)))
    # changes = get_substitutions_for_best_lineup(league, team_name, week)

    # for position, better_player, actual_player  in changes:
    #     print(team_name + " should have started " + better_player.name + " over " + actual_player.name + " at " + position)
    # print()



Name:  TaylorMade 3
Missed points:  5.85
Name:  Rigged AF
Missed points:  7.2
Name:  Injury Prone
Missed points:  21.6
Name:  jack goff
Missed points:  20.7
Name:  Chubby Diggs
Missed points:  33.2
Name:  Don't Puka The Bear
Missed points:  1.0
Name:  How I Metcalf Your Mother
Missed points:  38.1
Name:  No Punt Intended
Missed points:  6.5
Name:  Christian McCarryMe
Missed points:  0.97
